# 03 - Integración: proyección de firmas bulk en scRNA-seq

Este notebook calcula module scores a partir de firmas de GSE232710 y las proyecta sobre las células individuales.


In [ ]:
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

FIG_DIR = '../results/figures/'
TABLE_DIR = '../results/tables/'
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TABLE_DIR, exist_ok=True)

adata = sc.read_h5ad('../data_processed/iHSC_scRNAseq_Ng2025_qc_norm.h5ad')

# Cargar lista de genes up regulados por RETA
reta_up = pd.read_csv('../results/tables/02_up_genes.txt', header=None)[0].tolist()
reta_up = [g for g in reta_up if g in adata.var_names]
print(f'Genes en RETA_up usados: {len(reta_up)}')

sc.tl.score_genes(adata, reta_up, score_name='RETA_up_score')

# UMAP con score
sc.pl.umap(adata, color=['RETA_up_score'], show=False)
plt.savefig(FIG_DIR + '03_umap_RETA_up_score.png', dpi=300)
plt.close()

# Score medio por cluster
if 'cluster' in adata.obs.columns:
    cluster_scores = adata.obs.groupby('cluster')['RETA_up_score'].mean()
    cluster_scores.to_csv(TABLE_DIR + '03_RETA_up_score_by_cluster.csv')

    # Heatmap simple
    df = cluster_scores.to_frame(name='RETA_up_score')
    sns.heatmap(df, cmap='magma', annot=True)
    plt.savefig(FIG_DIR + '03_heatmap_RETA_up_score_by_cluster.png', dpi=300)
    plt.close()
else:
    print('No se encontró columna cluster en adata.obs')
